In [24]:
def x3(x1, x2):
  return (5 + 20 * x1 - 3 * x2)/(-15)

print(x3(2, 5))

-2.0


In [25]:
def psi(x1, x2):
  return -0.273 + 0.839 * x1 + 0.134 * x2

print(psi(3, 9))

3.4499999999999997


In [26]:
import numpy as np
print(round(1/(1 + np.exp(-psi(3, 9))), 3))

0.969


#Задание

1. Используйте метод главных компонент для набора данных MNIST (данные загрузите из keras.datasets, train dataset объема 60000)

In [27]:
from keras.datasets import mnist
from sklearn.decomposition import PCA

(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

X_train = X_train.reshape(60000, 28*28)
# X_pred = X_pred.reshape(60000, -1)
print(X_train.shape)

(60000, 784)


In [28]:
pca = PCA(n_components=49, svd_solver='full')
pca.fit(X_train)
print(sum(pca.explained_variance_ratio_))

0.8214281186628802


In [29]:
X_centered = X_train - X_train.mean(axis=0)
weights = pca.components_
print(X_centered.shape, weights.shape)
z = pca.transform(X_train)
print(z.shape)

(60000, 784) (49, 784)
(60000, 49)


In [30]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(z, y_train, test_size=0.3, random_state=68)

In [31]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(criterion='gini',
                             min_samples_leaf=10,
                             max_depth=20,
                             n_estimators=10,
                             random_state=68)
rfc_ovr = OneVsRestClassifier(rfc).fit(X_train1, y_train1)
y_pred1 = rfc_ovr.predict(X_test1)

In [32]:
cl3 = 0
for id, i in enumerate(y_pred1):
  if i == y_test1[id] == 3:
    cl3 += 1
print(cl3)

1591


In [33]:
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression(solver='lbfgs', random_state=68)

lrc_ovr = OneVsRestClassifier(lrc).fit(X_train1, y_train1)
y_pred2 = lrc_ovr.predict(X_test1)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
cl8 = 0
for id, i in enumerate(y_pred2):
  if i == y_test1[id] == 8:
    cl8 += 1
print(cl8)

1495


In [35]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='gini',
                             min_samples_leaf=10,
                             max_depth=20,
                             random_state=68)

dtc_ovr = OneVsRestClassifier(dtc).fit(X_train1, y_train1)
y_pred3 = dtc_ovr.predict(X_test1)

In [36]:
cl5 = 0
for id, i in enumerate(y_pred3):
  if i == y_test1[id] == 5:
    cl5 += 1
print(cl5)

1198


In [37]:
import pandas as pd

df = pd.read_csv('pred_for_task.csv')

fnames = df['FileName']
y_new = np.array(df['Label'])
X_new = np.array(df.drop(['FileName', 'Label'], axis = 1))

In [46]:
z_new = pca.transform(X_new)
print(fnames[17], round(rfc_ovr.predict_proba(z_new)[17][7], 3), y_new[17])
print(fnames[14], round(lrc_ovr.predict_proba(z_new)[14][1], 3), y_new[14])
print(fnames[18], round(dtc_ovr.predict_proba(z_new)[18][3], 3), y_new[18])

file18 0.88 7
file15 0.902 1
file19 0.674 3
